In [1]:
import numpy as np
import cv2
import imageio

In [2]:
def rgb2gray(img):
    # the picture read by cv2 returns tuple (b,g,r) 
    b, g, r = cv2.split(img)
    # according to the formula
    res = (15 * b + 75 * g + 38 * r) / 128
    return res

def getenergymap(img):
    b, g, r = cv2.split(img)
    b = np.absolute(cv2.Scharr(b, -1, 1, 0)) + np.absolute(cv2.Scharr(b, -1, 0, 1))
    g = np.absolute(cv2.Scharr(g, -1, 1, 0)) + np.absolute(cv2.Scharr(g, -1, 0, 1))
    r = np.absolute(cv2.Scharr(r, -1, 1, 0)) + np.absolute(cv2.Scharr(r, -1, 0, 1))
    return b + g + r

def areaofgt(datafile):
    im = cv2.imread(datafile)
    arr = np.array(im)
    count = 0
    for row in range(arr.shape[0]):
        for col in range(arr.shape[1]):
            if arr[row][col].all() != 0:
                count += 1
    return count

def areaofimg(datafile):
    im = cv2.imread(datafile)
    arr = np.array(im)
    # vertical, horizional, area
    return arr.shape[0], arr.shape[1], arr.shape[0] * arr.shape[1]

In [3]:
def dynamicforward(energymap, img):
    # forward seam removing(dynamic programming):
    I = rgb2gray(img)
    m, n = energymap.shape
    res = np.copy(energymap)
    for row in range(1, m):
        for col in range(n):
            if col == 0:
                rightup = res[row - 1, col + 1] + np.abs(I[row, col + 1] - I[row, col]) + np.abs(I[row - 1, col] - I[row, col + 1])
                middleup = res[row - 1, col] + np.abs(I[row, col + 1] - I[row, col])
                res[row, col] = energymap[row, col] + min(rightup, middleup)
            elif col == n - 1:
                upleft = res[row - 1, col - 1] + np.abs(I[row, col] - I[row, col - 1]) + np.abs(I[row - 1, col] - I[row, col - 1])
                middleup = res[row - 1, col] + np.abs(I[row, col] - I[row, col - 1])
                res[row, col] = energymap[row, col] + min(upleft, middleup)
            else:
                upleft = res[row - 1, col - 1] + np.abs(I[row, col + 1] - I[row, col - 1]) + np.abs(I[row - 1, col] - I[row, col - 1])
                rightup = res[row - 1, col + 1] + np.abs(I[row, col + 1] - I[row, col - 1]) + np.abs(I[row - 1, col] - I[row, col + 1])
                middleup = res[row - 1, col] + np.abs(I[row, col + 1] - I[row, col - 1])
                res[row, col] = energymap[row, col] + min(upleft, rightup, middleup)
    return res

In [4]:
def minpath(finalmap):
    # use backtracking to get the path has the lowest energy
    m, n = finalmap.shape
    res = np.zeros((m,), dtype=np.uint32)
    res[-1] = np.argmin(finalmap[-1])
    for i in range(m - 2, -1, -1):
        if res[i + 1] == 0:
            res[i] = np.argmin(finalmap[i, : 2])
        elif res[i + 1] == (n - 1):
            res[i] = np.argmin(finalmap[i, n - 2: n]) + n - 2
        else:
            res[i] = np.argmin(finalmap[i, res[i + 1] - 1: res[i + 1] + 2]) + res[i + 1] - 1
    return res

In [5]:
def deleteminpath(input, seamline):
    # delete the path(seamline)
    m, n = input.shape[: 2]
    res = np.zeros((m, n - 1, 3))
    for row in range(m):
        col = seamline[row]
        res[row, :, 0] = np.delete(input[row, :, 0], [col])
        res[row, :, 1] = np.delete(input[row, :, 1], [col])
        res[row, :, 2] = np.delete(input[row, :, 2], [col])
    input = np.copy(res)
    return input

In [6]:
def deleteminpathforgt(gt, seamline):
    # we use data/gt as reference, so in order to keep consistency, we need to operate on gt. 
    m, n = gt.shape
    res = np.zeros((m, n - 1))
    for row in range(m):
        col = seamline[row]
        res[row, :] = np.delete(gt[row, :], [col])
    gt = np.copy(res)
    return gt

In [7]:
def transpose(image, mode):
    # print(image)
    m, n, ch = image.shape
    res = np.zeros((n, m, ch))
    if mode == 0:
        # transpose
        imageT = np.fliplr(image)
        for c in range(ch):
            for row in range(m):
                res[:, row, c] = imageT[row, :, c]
    else:
        for c in range(ch):
            for row in range(m):
                res[:, m - 1 - row, c] = image[row, :, c]
    return res

In [8]:
def transposeforgt(mask, mode):
    # print(mask)
    m, n = mask.shape
    res = np.zeros((n, m))
    if mode == 0:
        # transpose
        imageT = np.fliplr(mask)
        for row in range(m):
            res[:, row] = imageT[row, :]
    else:
        for row in range(m):
            res[:, m - 1 - row] = mask[row, :]
    return res

In [9]:
def reduce_col(num, mode, gt, dynamic, resize):
    # resize: the count of column/rows we need to remove
    # delete a column, we set the gt have the largest energy(1000)
    # we get energymap, then find the path has lowest energy, then delete it in both img and gt.
    for i in range(resize):
        energymap = getenergymap(dynamic)
        energymap[np.where(gt > 0)] *= 1000
        finalmap = dynamicforward(energymap, dynamic)
        seamline = minpath(finalmap)
        # print(len(seamline))
        # if statement is used to get gif(613.gif && 913.gif)
        if num == 6 or num == 9:
            temp = np.copy(dynamic)
            # gifpre is a function to store the picture in each step of seam removing
            gifpre(temp, seamline, mode, num, i)
        # get the img and gt after seam removing
        dynamic = deleteminpath(dynamic, seamline)
        gt = deleteminpathforgt(gt, seamline)
    return dynamic, gt

In [10]:
def SeamCarving(th, nwidth, nheight, input, mask):
    # th: represent the i-th, especially for 613.png && 913.png to get 613.gif && 913.gif
    # (nwidth, nheight): the shape of picture after seam removing
    # input: data/imgs, mask: data/gt
    imgs = cv2.imread(input).astype(np.float64)
    gt = cv2.imread(mask, 0).astype(np.float64)
    cal = np.copy(imgs)
    # print(type(cal))
    widthreduce = imgs.shape[0] - nwidth
    highreduce = imgs.shape[1] - nheight
    print(highreduce, widthreduce)
    # exception, return original
    if highreduce < 0 or widthreduce < 0:
        print(imgs.shape[0], imgs.shape[1], nwidth, nheight, highreduce, widthreduce)
        cv2.imwrite(input, imgs.astype(np.uint8))
    # column delete
    cal, gt = reduce_col(th, 0, gt, cal, highreduce)
    # print(type(cal))
    # transpose img and gt
    cal = transpose(cal, 0)
    gt = transposeforgt(gt, 0)
    # row delete
    cal, gt = reduce_col(th, 1, gt, cal, widthreduce)
    # transpose to return original shape
    cal = transpose(cal, 1)
    gt = transposeforgt(gt, 1)
    return cal

In [11]:
def gifpre(temp, seamline, mode, num, count):
    # count: the count-th step of seam removing
    # this function is to get the each frame of gif, the green line is carved.
    for v in range(len(seamline)):
        temp[v][seamline[v]][0] = 0
        temp[v][seamline[v]][1] = 255
        temp[v][seamline[v]][2] = 0
    if mode == 0:
        outfilename = "gifmake/{}col{}.png".format(num * 100 + 13, count)
    else:
        outfilename = "gifmake/{}row{}.png".format(num * 100 + 13, count)
        temp = transpose(temp, 1)
    # you can see the result in folder "gifmake"
    cv2.imwrite(outfilename, temp.astype(np.uint8))

In [17]:
for i in range(10):
    image = "../data/imgs/{}.png".format(i * 100 + 13)
    gt = "../data/gt/{}.png".format(i * 100 + 13)
    out = "carving/{}.png".format(i * 100 + 13)
    # print((cv2.imread(gt).astype(np.float64)).shape)
    s = areaofgt(gt)
    width, height, S = areaofimg(image)
    print(height, width)
    # in order to meet the requirement
    prop = S - s / 2
    newwidth = int(np.sqrt(width * prop / height))
    newheight = int(np.sqrt(height * prop / width))
    print(newheight, newwidth)
    res = SeamCarving(i, newwidth, newheight, image, gt)
    print(res.shape)
    # the result of 10 pictures are stored in folder carving
    cv2.imwrite(out, res.astype(np.uint8))
    print("finish {}".format(image))

200 160
198 158
2 2
(158, 198, 3)
finish ../data/imgs/13.png
200 133
186 124
14 9
(124, 186, 3)
finish ../data/imgs/113.png
175 116
168 111
7 5
(111, 168, 3)
finish ../data/imgs/213.png
200 134
194 130
6 4
(130, 194, 3)
finish ../data/imgs/313.png
200 144
187 134
13 10
(134, 187, 3)
finish ../data/imgs/413.png
200 133
191 127
9 6
(127, 191, 3)
finish ../data/imgs/513.png
200 133
174 115
26 18
(115, 174, 3)
finish ../data/imgs/613.png
200 117
191 111
9 6
(111, 191, 3)
finish ../data/imgs/713.png
200 200
194 194
6 6
(194, 194, 3)
finish ../data/imgs/813.png
200 133
188 125
12 8
(125, 188, 3)
finish ../data/imgs/913.png


In [18]:
# from here, concat each frame to get the gif which show the process of seam carving in 613.png && 913.png
frame = []
for i in range(26):
    frame.append(imageio.imread("gifmake/613col{}.png".format(i)))
for i in range(18):
    frame.append(imageio.imread("gifmake/613row{}.png".format(i)))
imageio.mimsave("finalres/613.gif", frame, 'GIF', duration=0.3)
frame = []
for i in range(12):
    frame.append(imageio.imread("gifmake/913col{}.png".format(i)))
for i in range(8):
    frame.append(imageio.imread("gifmake/913row{}.png".format(i)))
imageio.mimsave("finalres/913.gif", frame, 'GIF', duration=0.3)